In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
dataset = pd.read_csv("AutoXing.csv")
dataset = dataset[["dataset", "metric", "teams"]]
dataset = dataset.loc[dataset["teams"] > 0]

In [3]:
os.listdir("./result/")

['result-2019-12-11.csv']

In [4]:
res = pd.read_csv("./result/result-2019-12-11.csv")

In [5]:
res = res.loc[res["status"] == "success"]
res.index = range(len(res))
res.shape

(28, 7)

In [6]:
# os.listdir("db-kaggle/")

In [7]:
res

,Unnamed: 0,phisical,status,private_score,public_score,rank,time
0,kaggle-allstate_claims,hdfs,success,1126.57408,1115.4252800000002,1.0,44023.36909675598
1,kaggle-amazon_employee,hdfs,success,0.90617,0.91214,51.0,6577.889317035675
2,kaggle-avazu,hdfs,success,0.3899,0.39205,1.0,18014.41471886635
3,kaggle-bike_sharing_demand,hdfs,success,NaN,0.39913000000000004,1.0,15426.881854057312
4,kaggle-bio_response,hdfs,success,0.38059,0.41712,51.0,31064.05154776573
5,kaggle-box_office,hdfs,success,NaN,2.0941799999999997,1.0,3435.4214079380035
6,kaggle-dont_overfit,hdfs,success,0.855,0.872,51.0,196.0761206150055
7,kaggle-elo_merchant,hdfs,success,3.7221800000000003,3.8078199999999995,1.0,14384.092439889908
8,kaggle-facebook_human_robot,hdfs,success,0.91205,0.8839,51.0,1348.1911306381226
9,kaggle-get_kicked,hdfs,success,0.26103000000000004,0.25925,8.0,11850.41955947876


In [11]:
def get_rank(_list, cur, teams, metric):
    if metric in ["mae", "logloss", "rmsle", "rmse"]:
        for i in range(len(_list)):
            if cur < _list[i]:
                return i / teams, str(i) + "/" + str(int(teams))
    if metric in ["auc", "gini", "r2"]:
        for i in range(len(_list)):
            if cur > _list[i]:
                return i / len(_list), str(i) + "/" + str(len(_list))

In [14]:
rank_res = []
for i in range(len(res)):
    
    name = res.loc[i, 'Unnamed: 0'][7:]

    metric = dataset.loc[dataset["dataset"] == name, "metric"].values[0]
    teams = dataset.loc[dataset["dataset"] == name, "teams"].values[0]

    score = float(res.loc[i, 'private_score'])
    if not score > 0:
        score = float(res.loc[i, 'public_score'])
    

    name = name.replace("_", "-")
    if name == "bio-response":
        name = "bioresponse"
    if name == "facebook-human-robot":
        name = "facebook-recruiting-iv"
    if name == "get-kicked":
        name = "DontGetKicked"
    if name == "give-me":
        name = "GiveMeSomeCredit"
    if name == "spring-leaf":
        name = "springleaf"
    if name == "stumble-upon":
        name = "stumbleupon"
    lb = pd.read_csv("db-kaggle/" + [x for x in os.listdir("db-kaggle/") if name in x][0], sep = ";", header = -1)
    lb.columns = ["rank", "team", "score"]

    rank, str_rank = get_rank(list(lb['score']), score, teams, metric)
    
    print("name: {}; metric: {}; teams: {}; score: {}; rank: {}".format(name, metric, teams, score, rank))

    rank_res.append([name, metric, teams, score, rank, str_rank])

name: allstate-claims; metric: mae; teams: 3052.0; score: 1126.57408; rank: 0.29226736566186107
name: amazon-employee; metric: auc; teams: 1687.0; score: 0.90617; rank: 0.05390995260663507
name: avazu; metric: logloss; teams: 1604.0; score: 0.3899; rank: 0.08728179551122195
name: bike-sharing-demand; metric: rmsle; teams: 3251.0; score: 0.39913000000000004; rank: 0.06705629037219317
name: bioresponse; metric: logloss; teams: 699.0; score: 0.38059; rank: 0.04864091559370529
name: box-office; metric: rmsle; teams: 1398.0; score: 2.0941799999999997; rank: 0.449928469241774
name: dont-overfit; metric: auc; teams: 2330.0; score: 0.855; rank: 0.029767040552200173
name: elo-merchant; metric: rmse; teams: 4127.0; score: 3.7221800000000003; rank: 0.7458202083838139
name: facebook-recruiting-iv; metric: auc; teams: 985.0; score: 0.91205; rank: 0.33908629441624366
name: DontGetKicked; metric: gini; teams: 570.0; score: 0.26103000000000004; rank: 0.010507880910683012
name: GiveMeSomeCredit; metric

In [17]:
rank_res = pd.DataFrame(rank_res)
rank_res.columns = ["name", "metric", "teams", "score", "rank", "str_rank"]
rank_res

,name,metric,teams,score,rank,str_rank
0,allstate-claims,mae,3052.0,1.126574e+03,0.292267,892/3052
1,amazon-employee,auc,1687.0,9.061700e-01,0.053910,91/1688
2,avazu,logloss,1604.0,3.899000e-01,0.087282,140/1604
3,bike-sharing-demand,rmsle,3251.0,3.991300e-01,0.067056,218/3251
4,bioresponse,logloss,699.0,3.805900e-01,0.048641,34/699
5,box-office,rmsle,1398.0,2.094180e+00,0.449928,629/1398
6,dont-overfit,auc,2330.0,8.550000e-01,0.029767,69/2318
7,elo-merchant,rmse,4127.0,3.722180e+00,0.745820,3078/4127
8,facebook-recruiting-iv,auc,985.0,9.120500e-01,0.339086,334/985
9,DontGetKicked,gini,570.0,2.610300e-01,0.010508,6/571


In [24]:
rank_res = rank_res.sort_values(by = "rank")
rank_res.index = range(len(rank_res))
rank_res["acc_avg_rank"] = rank_res['rank'].expanding().mean()
rank_res

,name,metric,teams,score,rank,str_rank,acc_avg_rank
0,mercedes-benz,r2,3831.0,5.538100e-01,0.003920,15/3827,0.003920
1,safe-driver,gini,5163.0,2.910500e-01,0.006977,36/5160,0.005448
2,GiveMeSomeCredit,auc,924.0,8.689400e-01,0.009730,9/925,0.006875
3,DontGetKicked,gini,570.0,2.610300e-01,0.010508,6/571,0.007783
4,kobe-bryant,logloss,1117.0,5.988000e-01,0.014324,16/1117,0.009092
5,social-network,auc,132.0,8.772100e-01,0.022222,3/135,0.011280
6,dont-overfit,auc,2330.0,8.550000e-01,0.029767,69/2318,0.013921
7,bioresponse,logloss,699.0,3.805900e-01,0.048641,34/699,0.018261
8,amazon-employee,auc,1687.0,9.061700e-01,0.053910,91/1688,0.022222
9,bike-sharing-demand,rmsle,3251.0,3.991300e-01,0.067056,218/3251,0.026705


In [26]:
rank_res.to_csv("./output/20191211.csv", index = False)